In [1]:
import torch
import torch.nn.functional as F
import math
import torch.nn.functional as F
from torch import nn, Tensor, optim
from torch.utils.data import TensorDataset, dataset
from torch.utils.data import DataLoader
from pathlib import Path
import os
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("CUDA Available: ", dev)


path = Path(os.getcwd()).parent.parent
path = path/'data/Titanic'
path


pd.options.display.float_format = '{:.2f}'.format
torch.manual_seed(42)

#read Data
data = pd.read_csv(path/'train.csv')
data.head()
dataCopy = data.copy()
dataCopy.isna().sum()

#fill null columns:
modes = dataCopy.mode().iloc[0]
dataCopy.fillna(modes, inplace=True)
dataCopy.isna().sum()

#create Dummy Columsn for non numaric columns:
#get non-numeric Columns:
dataCopy.describe(include=[object])

dataCopy['LogFare'] = np.log(dataCopy['Fare']+1)
dataCopy.head()

dataCopy = pd.get_dummies(dataCopy, columns=["Sex","Pclass","Embarked"])

CUDA Available:  cuda


In [3]:
indepn_col = dataCopy[['Age', 'SibSp', 'Parch', 'LogFare', 'Sex_male', 'Sex_female', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]
dpen_Col = dataCopy[['Survived']]
# indepn_col.head()
# dpen_Col.head()
indepn_col.shape

(891, 12)

In [4]:
#split into training data and validation data with SKlearn
indepenTrain, indepenVal, dpenTrain, dpenVal = train_test_split(indepn_col, dpen_Col, test_size=0.33, random_state=42)

#put pandasframes into Tensors:

indepenTrainTens = torch.tensor(indepenTrain.values, device=dev, dtype=torch.float32)
dpenTrainTens = torch.tensor(dpenTrain.values, device=dev, dtype=torch.float32)
indepenValTens = torch.tensor(indepenVal.values, device=dev, dtype=torch.float32)
dpenValTens = torch.tensor(dpenVal.values, device=dev, dtype=torch.float32)

print(indepenTrainTens.dtype, dpenTrainTens.dtype)
print(indepenTrainTens.shape, dpenTrainTens.shape)
print(indepenValTens.shape, dpenValTens.shape)

#put tensors into pytorch datasets
train_ds = TensorDataset(indepenTrainTens, dpenTrainTens)
valid_ds = TensorDataset(indepenValTens, dpenValTens)

torch.float32 torch.float32
torch.Size([596, 12]) torch.Size([596, 1])
torch.Size([295, 12]) torch.Size([295, 1])


In [32]:
# tempx, TempY = train_ds[0:5]
# print(tempx.dtype, TempY.dtype)

In [5]:
class TitanicNN_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(12, 50)
        self.layer2 = nn.Linear(50, 25)
        self.layer3 = nn.Linear(25, 1)
        self.sigmoid1 = nn.Sigmoid()
        
    def forward(self, xb):
        layer1_out = F.relu(self.layer1(xb))
        layer2_out = F.relu(self.layer2(layer1_out))
        layer3_out = self.layer3(layer2_out)
        sigmoid_out = self.sigmoid1(layer3_out)

        return sigmoid_out

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)


def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    loss_list = []
    validate_loss_list = []
    

    for epoch in range(epochs):
        running_loss = 0
        running_num = 0
        model.train()
        for xb, yb in train_dl:
            losses, nums = loss_batch(model, loss_func, xb, yb, opt)
            running_loss += losses*nums
            running_num += nums
        
        trainingLoss = running_loss/running_num
        loss_list.append(trainingLoss)

        model.eval()

        with torch.no_grad():
           losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        validate_loss_list.append(val_loss)


        print(f'Epoch: {epoch} trainginLoss: {trainingLoss} ValidationLoss:{val_loss}')


    return validate_loss_list, loss_list
            


In [10]:
bs = 128
epochs = 2000
lr = .001

train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)


model = TitanicNN_Model()



model.to(dev)

optimizer = optim.SGD(model.parameters(), lr=lr)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=0.001)
 

# loss_func = F.cross_entropy
# loss_func = torch.nn.BCELoss()
loss_func = F.binary_cross_entropy

validate_loss_list, loss_list = fit(epochs, model, loss_func, optimizer, train_dl, valid_dl)

Epoch: 0 trainginLoss: 0.738819633154261 ValidationLoss:0.721544521946018
Epoch: 1 trainginLoss: 0.712376222114435 ValidationLoss:0.7029225022105847
Epoch: 2 trainginLoss: 0.6946079066935802 ValidationLoss:0.691653627864385
Epoch: 3 trainginLoss: 0.6834909047856427 ValidationLoss:0.6841449814327693
Epoch: 4 trainginLoss: 0.6761639862252562 ValidationLoss:0.6795274423340619
Epoch: 5 trainginLoss: 0.670770641541321 ValidationLoss:0.6768393362982799
Epoch: 6 trainginLoss: 0.6676551363612181 ValidationLoss:0.6750326601125426
Epoch: 7 trainginLoss: 0.6651903890923365 ValidationLoss:0.6739583872132382
Epoch: 8 trainginLoss: 0.6637296644633248 ValidationLoss:0.6733471143043648
Epoch: 9 trainginLoss: 0.6627753965006579 ValidationLoss:0.672955351764873
Epoch: 10 trainginLoss: 0.6619730759787079 ValidationLoss:0.6727242665775751
Epoch: 11 trainginLoss: 0.6615109431663616 ValidationLoss:0.6725636478197776
Epoch: 12 trainginLoss: 0.6612976537454849 ValidationLoss:0.6724779405836332
Epoch: 13 train

In [11]:
validationDF = pd.DataFrame({'trainLoss': loss_list,'validationLoss': validate_loss_list})
validationDF.index.name = 'epoch'

pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

fig = px.line(validationDF, labels=dict(
    created_at="Date", value="Validation Loss", variable="value"
))
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()

In [ ]:
pred = model(xbTemp)

In [15]:
xbTemp,ybtemp = train_ds[0:]
xbTemp.shape
pred = model(xbTemp)
pred.shape

torch.Size([596, 1])

In [ ]:
# test_features = torch.tensor(test_scaled.to_numpy())
 
# test_prediction = model(test_features.float()).detach().numpy().flatten()
 
# test_prediction_binary = (test_prediction > 0.5).astype(np.int)
 
# test_prediction_df = pd.DataFrame(test_prediction_binary,
#                                   index=test.index,
#                                   columns=["Survived"])





# plt.plot(loss_list, linewidth=3)
# plt.plot(validate_loss_list, linewidth=3)
# plt.legend(("Training Loss", "Validation Loss"))
# plt.xlabel("Epoch")
# plt.ylabel("BCE Loss")

# print(epoch, val_loss)

In [ ]:
# model = torch.nn.Sequential(torch.nn.Linear(12, 50),
#                             torch.nn.ReLU(),
#                             torch.nn.Linear(50, 1),
#                             torch.nn.Sigmoid())


# model[0].weight



